In [ ]:
# todo NOTE: this script does not reproduce original result since we removed device IDs

import pandas as pd
import numpy as np

## read generic category mapped files
file_manual_identified_devices = '../../Device Identification/Device Category Mapping/manual_device_category_merged.csv'
file_gpt_identified_devices = '../../Device Identification/Device Category Mapping/gpt_device_id_generic_category.csv'
file_target_IoT_device = '../../Device Identification/IoT_devices_v5.csv'

combined_iot_devices = pd.read_csv(file_target_IoT_device)[['device_id']]
manual_device_category_merged = pd.read_csv(file_manual_identified_devices).rename(
    columns={'generic_category':'man_generic_category',
             'vendor_name':'man_vendor_name',
             'device_category':'man_device_category'})
gpt_device_category_merged = pd.read_csv(file_gpt_identified_devices)[['device_id',
                                                    'gpt_clean_vendor',
                                                    'gpt_clean_type',
                                                    'generic_category']].rename(
    columns={'generic_category':'gpt_generic_category'})

# todo NOTE: the following line cause runtime/space error since we removed device IDS 
combined_iot_devices = pd.merge(combined_iot_devices, manual_device_category_merged, on='device_id', how='left')
combined_iot_devices = pd.merge(combined_iot_devices, gpt_device_category_merged, on='device_id', how='left')
combined_iot_devices.replace(np.NAN, '', inplace=True)


combined_iot_devices['product'] = combined_iot_devices.apply(lambda row: 
                                                             row.man_vendor_name.lower()+ ':'+ row.man_device_category.lower() if len(row.man_vendor_name.lower()+ ':'+ row.man_device_category.lower())>2 else row.gpt_clean_vendor.lower()+ ':'+ row.gpt_clean_type.lower(), axis=1)

print(combined_iot_devices['product'].nunique())